In [24]:
import numpy as np
import xarray as xr
import pandas as pd

from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

## To compute the linear regression we need data files:
- CMIP5_time_series.nc
- CMIP5_time_series_smoothed.nc
- CMIP5_time_series_smoothed_10yr.nc
- CMIP6_time_series.nc
- CMIP6_time_series_smoothed.nc
- CMIP6_time_series_smoothed_10yr.nc

In the next cell you can choose either the smoothed or unsmoothed time series.

In [35]:
data_choice = 'Smoothed'

if data_choice == 'Unsmoothed':
    dsCMIP5 = xr.open_dataset('Data/CMIP5_time_series.nc')
    dsCMIP6 = xr.open_dataset('Data/CMIP6_time_series.nc')

elif data_choice == 'Smoothed':
    dsCMIP5 = xr.open_dataset('Data/CMIP5_time_series_smoothed.nc')
    dsCMIP6 = xr.open_dataset('Data/CMIP6_time_series_smoothed.nc')

elif data_choice == 'Smoothed_10yr':
    dsCMIP5 = xr.open_dataset('Data/CMIP5_time_series_smoothed_10yr.nc')
    dsCMIP6 = xr.open_dataset('Data/CMIP6_time_series_smoothed_10yr.nc')

else:
    print('data_choice is not specified correctly. Choose `Smoothed`, `Smoothed_10yr` or `Unsmoothed`')

## Functions for linear regression:
- lin_reg_multi(varx, vary)

In [27]:
def select_models(ds, varx1, vary, check, varx2=None):
    '''
    Check is either 'model', or 'scenario'
    Find models for which both variables (varx1, vary) are available
    Input is an xarray dataset (CMIP5ds, CMIP5ds_LF, CMIP6ds, CMIP6ds_LF). 
    If you want to check the scenarios per model, already select the model in the input: eg. CMIP5ds.sel(model=mod)
    It returns the model/sce names of models/sces that have data for all three variables and returns the new dataset.
    '''
    x1 = ds[varx1].dropna(check,'all')[check].values
    y = ds[vary].dropna(check,'all')[check].values
    
    if varx2 == None:
        check_list = np.sort(list(set(x1)&set(y)))

    if varx2 is not None:
        x2 = ds[varx2].dropna(check,'all')[check].values
        check_list = np.sort(list(set(x1)&set(x2)&set(y)))
    
    if check == 'model':
        ds_new = ds.sel(model=check_list)
        
    elif check == 'scenario':
        ds_new = ds.sel(scenario=check_list)
    
    return check_list, ds_new

In [28]:
#%% Multi-linear regression
def lin_reg_multi(varx, vary):
    regr = linear_model.LinearRegression()

    varx = varx.dropna()
    vary = vary.dropna()

    regr.fit(varx, vary)
    
    vary_pred = regr.predict(varx)

    mse = mean_squared_error(vary, vary_pred)
    r2 = r2_score(vary, vary_pred)
    slope = regr.coef_
    intercept = regr.intercept_
    
    return vary_pred, mse, r2, slope, intercept

In [29]:
def lin_fit(ds, varx, mip):    
    ds_new = select_models(ds, varx, 'zos', 'model')[1]
    
    nan_array_sce = np.ones(95)*np.nan
    mods, sces = [],[]
    dfs = []
    mses, r2s, slope_varx, intercepts = [],[],[],[]
    
    for j, mod in enumerate(ds_new.model.values):
        
        if mod == 'GFDL-CM3':           # model does not have zos available for same scenarios as one of the regressors
            pass 
        else:
            sces_in_mod = select_models(ds_new.sel(model=mod), varx, 'zos', 'scenario')[0]
            ds_mod = ds_new.sel(model=mod)
            mods.append(mod)           # append model name
            sces.append(sces_in_mod)   # append scenarios available for this model
        
            if mip == 'CMIP5': 
                end_hist = 2005
                nan_num = 95
                sce_names = ['rcp26','rcp45','rcp85']

            elif mip == 'CMIP6':
                end_hist = 2014
                nan_num = 86
                sce_names = ['ssp126','ssp245','ssp585']

            ds_hist = ds_mod.sel(time=slice(1900,end_hist),scenario=sces_in_mod[0])  # select historical period for one sce
            ds_sces = ds_mod.sel(time=slice(end_hist,2100))                          # select future period for all sces

            DSL_mod, varx_mod = ds_hist.zos.values, ds_hist[varx].values      
        
            for sce in sce_names:        
                ds_sce = ds_sces.sel(scenario=sce)
            
                zos = ds_sce.zos.values
                varx_list = ds_sce[varx].values
            
                if(np.isnan(zos).any()) or (np.isnan(varx_list).any()):
                    DSL_mod = np.append(DSL_mod, np.ones(nan_num)*np.nan)
                    varx_mod = np.append(varx_mod, np.ones(nan_num)*np.nan)
                else:
                    DSL_mod = np.append(DSL_mod, zos)
                    varx_mod = np.append(varx_mod, varx_list)
            
            # Store total data in dataframe for each model
            d = {'DSL': DSL_mod, 'varx': varx_mod}
            df = pd.DataFrame(data=d)    
            dfs.append(df)
         
            # Compute regression for this model    
            X = df[['varx']]
            Y = df[['DSL']]
        
            #print(df)
            linreg = lin_reg_multi(X,Y)
            mse, r2, slope, intercept = linreg[1], linreg[2], linreg[3], linreg[4]
        
            mses.append(mse)
            r2s.append(r2)
            slope_varx.append(slope[0][0])
            intercepts.append(intercept[0])

    # Construct dataframe to store parameter values
    d = {'model': mods, 'sces':sces, 'r2-score':r2s, 'alpha': intercepts, 'beta':slope_varx, 'mse': mses}
    df_params = pd.DataFrame(data=d).set_index('model').round(4)
            
    return dfs, df_params

In [30]:
def multi_lin_fit(ds, varx1, varx2, mip):
    
    ds_new = select_models(ds, varx1, 'zos','model',varx2)[1]
    
    nan_array_sce = np.ones(95)*np.nan
    mods, sces = [],[]
    dfs = []
    mses, r2s, slope_varx1, slope_varx2, intercepts = [],[],[],[],[]
    
    for j, mod in enumerate(ds_new.model.values):
        sces_in_mod = select_models(ds_new.sel(model=mod), varx1, 'zos', 'scenario', varx2)[0]
        ds_mod = ds_new.sel(model=mod)
        mods.append(mod)           # append model name
        sces.append(sces_in_mod)   # append scenarios available for this model
        
        if mip == 'CMIP5': 
            end_hist = 2005
            nan_num = 95
            sce_names = ['rcp26','rcp45','rcp85']

        elif mip == 'CMIP6':
            end_hist = 2014
            nan_num = 86
            sce_names = ['ssp126','ssp245','ssp585']
                
        ds_hist = ds_mod.sel(time=slice(1900,end_hist),scenario=sces_in_mod[0])  # select historical period for one sce
        ds_sces = ds_mod.sel(time=slice(end_hist,2100))                          # select future period for all sces

        DSL_mod, varx1_mod, varx2_mod = ds_hist.zos.values, ds_hist[varx1].values, ds_hist[varx2].values        
        
            
        for sce in sce_names:        
            ds_sce = ds_sces.sel(scenario=sce)
            
            zos = ds_sce.zos.values
            varx1_list = ds_sce[varx1].values
            varx2_list = ds_sce[varx2].values
            
            if(np.isnan(zos).any()) or (np.isnan(varx1_list).any()) or (np.isnan(varx2_list).any()):
                DSL_mod = np.append(DSL_mod, np.ones(nan_num)*np.nan)
                varx1_mod = np.append(varx1_mod, np.ones(nan_num)*np.nan)
                varx2_mod = np.append(varx2_mod, np.ones(nan_num)*np.nan)
            else:
                DSL_mod = np.append(DSL_mod, zos)
                varx1_mod = np.append(varx1_mod, varx1_list)
                varx2_mod = np.append(varx2_mod, varx2_list)
            
        # Store total data in dataframe for each model
        d = {'DSL': DSL_mod, 'varx1': varx1_mod, 'varx2': varx2_mod}
        df = pd.DataFrame(data=d)    
        dfs.append(df)
         
        # Compute regression for this model    
        X = df[['varx1','varx2']] 
        Y = df[['DSL']] 
        
        #print(df)
        linreg = lin_reg_multi(X,Y)
        mse, r2, slope, intercept = linreg[1], linreg[2], linreg[3], linreg[4]
        
        mses.append(mse)
        r2s.append(r2)
        slope_varx1.append(slope[0][0])
        slope_varx2.append(slope[0][1])
        intercepts.append(intercept[0])
    
        
    # Construct dataframe to store parameter values
    d = {'model': mods, 'sces':sces, 'r2-score':r2s,'alpha': intercepts, 'beta1':slope_varx1, 'beta2': slope_varx2, 'mse': mses}
    df_params = pd.DataFrame(data=d).set_index('model').round(4)
    
            
    return dfs, df_params

## Single regression

In [31]:
LR_CMIP5_GSAT_ts,   LR_CMIP5_GSAT    = lin_fit(dsCMIP5, 'GSAT',   'CMIP5')
LR_CMIP5_GMTSL_ts,  LR_CMIP5_GMTSL   = lin_fit(dsCMIP5, 'GMTSL',  'CMIP5')
LR_CMIP5_AMOC35_ts, LR_CMIP5_AMOC35  = lin_fit(dsCMIP5, 'AMOC35', 'CMIP5')
LR_CMIP5_AMOC26_ts, LR_CMIP5_AMOC26  = lin_fit(dsCMIP5, 'AMOC26', 'CMIP5')

LR_CMIP6_GSAT_ts,   LR_CMIP6_GSAT    = lin_fit(dsCMIP6, 'GSAT',   'CMIP6')
LR_CMIP6_GMTSL_ts,  LR_CMIP6_GMTSL   = lin_fit(dsCMIP6, 'GMTSL',  'CMIP6')
LR_CMIP6_AMOC35_ts, LR_CMIP6_AMOC35  = lin_fit(dsCMIP6, 'AMOC35', 'CMIP6')
LR_CMIP6_AMOC26_ts, LR_CMIP6_AMOC26  = lin_fit(dsCMIP6, 'AMOC26', 'CMIP6')

In [32]:
# Save parameters to Results_LR
LR_CMIP5_GSAT.to_csv('Results_LR/LR_CMIP5_GSAT.csv')
LR_CMIP5_GMTSL.to_csv('Results_LR/LR_CMIP5_GMTSL.csv')
LR_CMIP5_AMOC35.to_csv('Results_LR/LR_CMIP5_AMOC35.csv')
LR_CMIP5_AMOC26.to_csv('Results_LR/LR_CMIP5_AMOC26.csv')

LR_CMIP6_GSAT.to_csv('Results_LR/LR_CMIP6_GSAT.csv')
LR_CMIP6_GMTSL.to_csv('Results_LR/LR_CMIP6_GMTSL.csv')
LR_CMIP6_AMOC35.to_csv('Results_LR/LR_CMIP6_AMOC35.csv')
LR_CMIP6_AMOC26.to_csv('Results_LR/LR_CMIP6_AMOC26.csv')

## Multiple regression

In [33]:
LR_CMIP5_GSAT_GMTSL     = multi_lin_fit(dsCMIP5, 'GSAT',  'GMTSL',  'CMIP5')[1]
LR_CMIP5_GSAT_AMOC35    = multi_lin_fit(dsCMIP5, 'GSAT',  'AMOC35', 'CMIP5')[1]
LR_CMIP5_GSAT_AMOC26    = multi_lin_fit(dsCMIP5, 'GSAT',  'AMOC26', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC35   = multi_lin_fit(dsCMIP5, 'GMTSL', 'AMOC35', 'CMIP5')[1]
LR_CMIP5_GMTSL_AMOC26   = multi_lin_fit(dsCMIP5, 'GMTSL', 'AMOC26', 'CMIP5')[1]

LR_CMIP6_GSAT_GMTSL     = multi_lin_fit(dsCMIP6, 'GSAT',  'GMTSL',  'CMIP6')[1]
LR_CMIP6_GSAT_AMOC35    = multi_lin_fit(dsCMIP6, 'GSAT',  'AMOC35', 'CMIP6')[1]
LR_CMIP6_GSAT_AMOC26    = multi_lin_fit(dsCMIP6, 'GSAT',  'AMOC26', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC35   = multi_lin_fit(dsCMIP6, 'GMTSL', 'AMOC35', 'CMIP6')[1]
LR_CMIP6_GMTSL_AMOC26   = multi_lin_fit(dsCMIP6, 'GMTSL', 'AMOC26', 'CMIP6')[1]

In [34]:
LR_CMIP5_GSAT_GMTSL.to_csv('Results_LR/LR_CMIP5_GSAT_GMTSL.csv')
LR_CMIP5_GSAT_AMOC35.to_csv('Results_LR/LR_CMIP5_GSAT_AMOC35.csv')
LR_CMIP5_GSAT_AMOC26.to_csv('Results_LR/LR_CMIP5_GSAT_AMOC26.csv')
LR_CMIP5_GMTSL_AMOC35.to_csv('Results_LR/LR_CMIP5_GMTSL_AMOC35.csv')
LR_CMIP5_GMTSL_AMOC26.to_csv('Results_LR/LR_CMIP5_GMTSL_AMOC26.csv')

LR_CMIP6_GSAT_GMTSL.to_csv('Results_LR/LR_CMIP6_GSAT_GMTSL.csv')
LR_CMIP6_GSAT_AMOC35.to_csv('Results_LR/LR_CMIP6_GSAT_AMOC35.csv')
LR_CMIP6_GSAT_AMOC26.to_csv('Results_LR/LR_CMIP6_GSAT_AMOC26.csv')
LR_CMIP6_GMTSL_AMOC35.to_csv('Results_LR/LR_CMIP6_GMTSL_AMOC35.csv')
LR_CMIP6_GMTSL_AMOC26.to_csv('Results_LR/LR_CMIP6_GMTSL_AMOC26.csv')